# Utils

In [174]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 7]
import scipy.stats as sts
import glob
from collections import defaultdict


def plot_data(path, key='VanillaAvgRet'):
    xlabel='Iteration'
    df = pd.read_csv(path)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    fig.suptitle(key)
    mean = df[key]
    ax.plot(df[xlabel], mean)
    return fig


In [132]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue May  1 11:57:26 2018

@author: matteo
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts


def read_data(path, iters=None, default_batchsize=100, scale='Eps'):
    df = pd.read_csv(path, encoding='utf-8')
    if iters: df = df.loc[:iters, :]
    if not 'BatchSize' in df: df['BatchSize'] = default_batchsize
    if not 'AvgRet' in df: df['AvgRet'] = df['AverageReturn']
    if not 'EpsThisIter' in df: df['EpsThisIter'] = df['BatchSize']
    df['EpsSoFar'] = np.cumsum(df['EpsThisIter'])
    if 'SamplesThisIter' in df: df['SamplesSoFar'] = np.cumsum(df['SamplesThisIter'])
    df['CumAvgRet'] = np.cumsum(df['AvgRet']*df[scale+'ThisIter'])/np.sum(df[scale+'ThisIter'])
    return df

def moments(dfs):
    concat_df = pd.concat(dfs, axis=1)
    mean_df = pd.concat(dfs, axis=1).groupby(by=concat_df.columns, axis=1).mean()
    std_df = pd.concat(dfs, axis=1).groupby(by=concat_df.columns, axis=1).std()
    return mean_df, std_df

def plot_all(dfs, key='AvgRet', ylim=None, scale='Samples'):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for df in dfs:
        value = df[key]
        ax.plot(df[scale+'SoFar'], value)
    return fig

def plot_ci(dfs, conf=0.95, key='ReturnLastEpisode', ylim=None, scale='Iteration'):
    n_runs = len(dfs)
    mean_df, std_df = moments(dfs)
    mean = mean_df[key]
    std = std_df[key]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(mean_df[scale], mean)
    interval = sts.t.interval(conf, n_runs-1,loc=mean,scale=std/np.sqrt(n_runs))
    ax.fill_between(mean_df[scale], interval[0], interval[1], alpha=0.3)
    if ylim: ax.set_ylim(ylim)
    return fig

def compare(candidates, conf=0.95, key='AvgRet', ylim=None, xlim=None, scale='Eps'):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    entries = []
    for candidate_name in candidates:
        entries.append(candidate_name)
        dfs = candidates[candidate_name]
        n_runs = len(dfs)
        mean_df, std_df = moments(dfs)
        mean = mean_df[key]
        std = std_df[key]
        ax.plot(mean_df[scale], mean)
        interval = sts.t.interval(conf, n_runs-1,loc=mean,scale=std/np.sqrt(n_runs))
        ax.fill_between(mean_df[scale+'SoFar'], interval[0], interval[1], alpha=0.3)
        print(candidate_name, end=': ')
        print_ci(dfs, conf)
    ax.legend(entries, loc='upper left')
    if ylim: ax.set_ylim(ylim)
    if xlim: ax.set_xlim(xlim)
    return fig

def compare_2(means, stds, conf, ylim=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for mean, std in zip(means, stds):
        n_runs = len(mean)
        ax.plot(mean.index, mean)
        interval = sts.t.interval(conf, n_runs-1,loc=mean,scale=std/np.sqrt(n_runs))
        ax.fill_between(mean.index, interval[0], interval[1], alpha=0.3)
    if ylim: ax.set_ylim(ylim)
    ax.legend(bounds)
    return fig

def plot_data(path, key='VanillaAvgRet'):
    df = pd.read_csv(path)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    mean = df[key]
    ax.plot(df['EpsSoFar'], mean)
    return fig

def print_ci(dfs, conf=0.95, key='CumAvgRet'):
    n_runs = len(dfs)
    mean_df, std_df = moments(dfs)
    total_horizon = np.sum(mean_df['AvgEpLen'])
    mean = mean_df[key][len(mean_df)-1]
    std = std_df[key][len(mean_df)-1]
    interval = sts.t.interval(conf, n_runs-1,loc=mean,scale=std/np.sqrt(n_runs))
    print('%f \u00B1 %f\t[%f, %f]\t total horizon: %d' % (mean, std, interval[0], interval[1], int(total_horizon)))

def save_ci(dfs, conf=0.95, key='AvgRet', ylim=None, scale='Eps', horizon=500, filename='prova.csv'):
    n_runs = len(dfs)
    mean_df, std_df = moments(dfs)
    mean = mean_df[key]
    std = std_df[key]
    interval = sts.t.interval(conf, n_runs-1,loc=mean,scale=std/np.sqrt(n_runs))
    low = pd.Series(interval[0])
    high = pd.Series(interval[1])
    iteration = pd.Series(horizon*np.array(range(len(mean))))
    df_out = pd.DataFrame(list(zip(iteration, mean, std, low, high)), columns=['iteration', 'mean', 'std', 'low', 'up'])
    df_out.to_csv(filename, index=False)


In [191]:
def read_csv(path, substrings, max_iters=None):
    
    # Getting a list of relevant filenames
    filenames = glob.glob(path)
    if substrings is not None:
        for substring in substrings:
            filenames = [f for f in filenames if substring in f ]
    # Turning them into dfs, dropping non-numeric columns
    dfs_list = [pd.read_csv(filename).select_dtypes(['number']) for filename in filenames]
    # Make sure that all dfs have the same number of trajectories
    if max_iters is not None:
        dfs_list = [df[:max_iters] for df in dfs_list]
    return dfs_list, filenames
    
    
def create_average_df(path, substrings=None, max_iters=None, kpis=None):
    
    dfs_list, filenames = read_csv(path, substrings, max_iters)
    # Adding the filename to each dataframe
    for df, filename in zip(dfs_list, filenames):
      df['Filename'] = filename
    # Combine the list of dataframes, on top of each other
    combo_df = pd.concat(dfs_list, ignore_index=True, sort=False)
    if kpis is None:
        # Set kpis equal to the list of column names except 'Filename'
        kpis = combo_df.columns[:-1]
    n_files = len(filenames)
    means = defaultdict(list)
    for kpi in kpis:
        for it in range(max_iters):
            means[kpi].append(np.array([combo_df[kpi][it + f*max_iters] for f in range(n_files)]).mean())
    df_out = pd.DataFrame.from_dict(means)
    return df_out, filenames, combo_df, dfs_list

In [192]:
path = "./old_logs/which_dtheta/which_dtheta_fixed_*"
max_iters = 100
substrings = ['delta=0.1', 'dtheta=0.1']
kpis = ['ReturnLastEpisode', 'Bound']
df, filenames, combo_df, dfs_list = create_average_df(path, substrings, max_iters, kpis)

In [193]:
mean_df, std_df = moments(dfs_list)

DataError: No numeric types to aggregate

In [194]:
mean_dfs, std_dfs = [], []
all_dtheta = [1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.005, 0.0001]
for dtheta in [1, 0.5, 0.1, 0.05, 0.01, 0.005]:
    s = 'dtheta={}'.format(dtheta)
    substrings = ['delta=0.1', s]
    df, filenames, combo_df, dfs_list = create_average_df(path, substrings, max_iters, kpis)
    mean_df, std_df = moments(dfs_list)
    mean_dfs.append(mean_df['ReturnMean'])
    std_dfs.append(std_df['ReturnMean'])
    

DataError: No numeric types to aggregate

In [ ]:
conf = 0.95
bounds = range(6)
fig = compare_2(mean_dfs, std_dfs, conf)

In [195]:
filenames

['./old_logs/which_dtheta/which_dtheta_fixed__LQG1D-V0_delta=0.1_seed=2_dtheta=1.0_20-31-39_5122018_27066.csv',
 './old_logs/which_dtheta/which_dtheta_fixed__LQG1D-V0_delta=0.1_seed=1_dtheta=1.0_20-31-40_5122018_87275.csv',
 './old_logs/which_dtheta/which_dtheta_fixed__LQG1D-V0_delta=0.1_seed=0_dtheta=1.0_20-31-40_5122018_10626.csv']

In [196]:

for dtheta in [1, 0.1, 0.01, 0.001]:
    s = 'dtheta={}'.format(dtheta)
    substrings = ['delta=0.1', s]
    df, filenames, combo_df, dfs_list = create_average_df(path, substrings, max_iters, kpis)
    plot_ci(dfs_list, key='ReturnMean')
    

DataError: No numeric types to aggregate

In [ ]:
substrings

In [165]:
dd = []
for i in range(len(dfs_list)):
    dd.append(dfs_list[i]['ReturnLastEpisode'])
np.array(dd)[:, :5]

array([[15.80704 ,  8.608342,  8.519454,  8.514177,  8.4897  ],
       [15.644508,  8.668443,  8.578568,  8.554674,  8.543934],
       [15.058442, 16.432243, 16.591774, 16.632465, 16.5498  ]])